In [1]:
import re
import sys
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import nltk
from datetime import datetime, timezone
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import demoji

In [2]:
sys.path.append("..")

In [3]:
from src.data.data_cleaning_module import *

In [4]:
nltk.download("rslp");

[nltk_data] Downloading package rslp to /home/gus/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [5]:
columns = ["tweet_text", "sentiment"]

In [6]:
# def stemming(texto: str) -> str:
#     stemmer = nltk.stem.RSLPStemmer()
#     palavras = []
#     palavras = " ".join([stemmer.stem(palavra) for palavra in texto.split()])
#     return palavras

# def lematization(texto: str) -> str:
#     nlp = spacy.load("pt_core_news_sm")
#     doc = nlp(texto)
#     texto = " ".join([token.lemma_ for token in doc])
#     return texto


# def remover_stop_words(texto: str) -> str:
#     stopwords = nltk.corpus.stopwords.words("portuguese")
#     print(stopwords)
#     texto = " ".join(list(filter(lambda x: x not in stopwords, texto.split())))
#     return texto


# def formatar_texto(texto: str) -> str:
#     texto = (
#         re.sub(r"(http\S+)|(@\w+)", "", texto)  # remove links, usuários #
#         .replace(".", "")
#         .replace(";", "")
#         .replace("—", "")
#     )

#     texto = re.sub(r"(  +)", " ", texto)  # remove espaços duplos
#     texto = texto.lower().strip()

#     return texto

## Leitura de dados brutos (~700k tweets)

In [7]:
df_raw = pd.read_csv("../data/raw/NoThemeTweets.csv", delimiter = ",", usecols = columns)

### Seleção de 150.000 tweets de forma pseudo aleatória para poupar custo computacional, as proporções de tweets positivos e negativos são mantidas.
#### Essa será a nossa base de dados reduzida.

In [8]:
df_raw = df_raw.sample(150000)

### Checando as informações dos nossos dados com o metodo .info()

### Remoção de textos duplicados

In [9]:
df_raw.drop_duplicates(["tweet_text"], inplace=True)

### Aplicação de funções para remover links, nomes de usuários dos tweets e caracteres especiais.

In [10]:
df_raw["tweet_text"] = df_raw["tweet_text"].apply(
    lambda tweet: formatar_texto(texto=tweet))

In [11]:
df_raw["tweet_text"] = df_raw["tweet_text"].apply(
    lambda tweet: remove_special_chars(tweet))

### Remoção dos emojis dos textos

In [12]:
df_raw["tweet_text"] = df_raw["tweet_text"].apply(
    lambda tweet: demoji.replace(tweet, "")); # custoso

### Aplicação de função para etiquetar os sentimentos positivos com o valor 1 e negativos com o valor 0

In [13]:
df_raw["sentiment"] = df_raw["sentiment"].replace({"Positivo": 1, "Negativo": 0})

### Adicionando uma coluna com o número de palavras por tweets

In [15]:
df_raw = df_raw.assign(
    number_words=df_raw.tweet_text.apply(lambda x: len(x.split(" "))),
)  # adiciona coluna com número de palavras

### Remoção dos tweets que possuem menos de 5 palavras. Esses tweets curtos podem vir a atrapalhar a etapa de modelagem e portanto serão excluídos

In [16]:
formated_df_raw = df_raw.drop(
    df_raw[df_raw.number_words < 5].index
)  # remove tweets com menos de 5 palavras

### remoção da coluna coluna number_words que não é mais necessária

In [17]:
formated_df_raw.drop(["number_words"], axis=1, inplace=True)

In [18]:
del df_raw # liberando espaço na memória

# Etapa 2 - Geração de 4 DataFrames a partir da remoção ou não das Stopwords, e aplicação de Stemming ou Lematização. 
### DataFrame 1: sem Stopwords e Lematizado; 
### DataFrame 2 : sem Stopwords e com Stemming; 
### DataFrame 3: com Stopwords e Lematizado;
### DataFrame 4: com Stopwords e com Stimming.

<img src="images/Pipeline formatação de texto.png" width=auto heigth=auto>

In [21]:
df_no_stopwords = formated_df_raw.copy()

df_steamed_no_stopwords = formated_df_raw.copy()

df_steamed_no_stopwords["tweet_text"] = df_no_stopwords["tweet_text"].apply(
    lambda tweet: stemming(texto=tweet)
)

df_lemmetized_no_stopwords = formated_df_raw.copy()

df_lemmetized_no_stopwords["tweet_text"] = df_no_stopwords["tweet_text"].apply(
    lambda tweet: lematization(tweet)
)

### Checagem do DataFrame lematizado sem a remoção das stopwords

In [36]:
df_lemmetized_no_stopwords.head(10)

,tweet_text,sentiment
106869,af mas eu te amar tanto,0
156630,menina que ir passar um semana entristecir e e...,0
440072,esse treta de o cube só mostrar mais ainda o q...,0
373801,dormir demais né mas o dia passar voar mesmo n...,0
44344,hoje meu sócio falar que descobrir um novo for...,1
149059,mas eu não querer dormir se não eu acordo ce...,0
654378,my name is flora sem ter outro flora aqui qu...,1
478001,um goleada para começar,1
272211,muito obrigado por o palavra anjinho ah auhs...,1
634135,eu amar o talento de o menino de o shinee juro...,0


### Checagem do DataFrame com stemming sem a remoção das stopwords

In [23]:
df_steamed_no_stopwords.head(10)

,tweet_text,sentiment
106869,af mas eu te amo tant,0
156630,menin que vão pass uma seman entristec e em cl...,0
440072,ess tret da cub só mostr mais aind o quant é d...,0
373801,dorm demal né mas o dia pass vo mesm nem parec...,0
44344,hoj meu sóci fal que descobr uma nov form de c...,1
149059,mas eu não quer dorm se não eu acord ced e não...,0
654378,my nam is flor sem tem outr flor aqu que se ap...,1
478001,uma gole par começ,1
272211,muit obrig pel palavr anj ah auhsuahsahsaushu ...,1
634135,eu amo o talent do menin do shine jur pra voc ...,0


### Leitura das stopwords

In [26]:
stopwords = pd.read_csv("../data/no-theme-tweets/stopwords.csv");

### Remoção das stopwords e aplicação de Lematização e stemming.

In [29]:
df_with_stopwords = formated_df_raw.copy()

df_with_stopwords["tweet_text"] = df_with_stopwords["tweet_text"].apply(
    lambda tweet: remover_stop_words(tweet, stopwords)
)

df_steamed_with_stopwords = formated_df_raw.copy()

df_steamed_with_stopwords["tweet_text"] = df_with_stopwords["tweet_text"].apply(
    lambda tweet: stemming(texto=tweet)
)

df_lemmetized_with_stopwords = formated_df_raw.copy()

df_lemmetized_with_stopwords["tweet_text"] = df_with_stopwords["tweet_text"].apply(
    lambda tweet: lematization(tweet)
)

### Checagem do DataFrame com stemming e com remoção das stopwords

In [30]:
df_steamed_with_stopwords.head(10)

,tweet_text,sentiment
106869,af amo tant,0
156630,menin vão pass seman entristec clim desped,0
440072,tret cub mostr aind quant difícil pra idol ass...,0
373801,dorm demal né dia pass vo parec 18h falt pra p...,0
44344,hoj sóci fal descobr nov form comunic hardw vi...,1
149059,quer dorm acord ced consig madrug amanhã cas,0
654378,my nam is flor outr flor aqu apres agor cales ...,1
478001,gole começ,1
272211,obrig palavr anj ah auhsuahsahsaushu val esper...,1
634135,amo talent menin shine jur pra q qualqu músic ...,0


## checagem do dataframe lematizado e com remoção de stopwords

In [31]:
df_lemmetized_with_stopwords.head(10)

,tweet_text,sentiment
106869,af amo tanto,0
156630,menina ir passar semana entristecir clima desp...,0
440072,treta cube mostrar ainda quanto difícil pra id...,0
373801,dormir demais né dia passar voar parecer 18h f...,0
44344,hoje sócio falar descobrir novo forma comunica...,1
149059,querer dormir acordo cedo consigo madrugar ama...,0
654378,my name is flora outro flora aqui apresente ag...,1
478001,golear começar,1
272211,obrigado palavras anjinho ah auhsuahsahsaushua...,1
634135,amo talento menino shinee juro pra q qualquer ...,0


### checkagem da integridade do dados processados

In [33]:
print(df_steamed_no_stopwords.shape,df_lemmetized_no_stopwords.shape,df_steamed_with_stopwords.shape,df_lemmetized_with_stopwords.shape)

(123681, 2) (123681, 2) (123681, 2) (123681, 2)


### Salvando os DataFrames na pasta data/processed

In [34]:
df_steamed_no_stopwords.to_csv('../data/processed/df_steamed_no_stopwords.csv', index=False)
df_lemmetized_no_stopwords.to_csv('../data/processed/df_lemmetized_no_stopwords.csv', index=False)
df_steamed_with_stopwords.to_csv('../data/processed/df_steamed_with_stopwords.csv', index=False)
df_lemmetized_with_stopwords.to_csv('../data/processed/df_lemmetized_with_stopwords.csv', index=False)

In [35]:
!ls ../data/processed/

df_lemmetized_no_stopwords.csv	  df_steamed_no_stopwords.csv
df_lemmetized_with_stopwords.csv  df_steamed_with_stopwords.csv
